In [2]:
import sentencepiece as spm
import pandas as pd
import MeCab

## 조사 제거해서 문장 반환하는 함수

In [1]:
def Korean_tokenizer(x): 
    
    m = MeCab.Tagger()
    delete_tag = ['BOS/EOS', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']

    
    
    def remove_josa(sentence):
        sentence_split = sentence.split() # 원본 문장 띄어쓰기로 분리

        dict_list = []

        for token in sentence_split: # 띄어쓰기로 분리된 각 토큰 {'단어':'형태소 태그'} 와 같이 딕셔너리 생성
            m.parse('')
            node = m.parseToNode(token)
            word_list = []
            pos_list = []
        
            while node:
                morphs = node.feature.split(',')
                word_list.append(node.surface)
                pos_list.append(morphs[0])
                node = node.next
            dict_list.append(dict(zip(word_list, pos_list)))        

        for dic in dict_list: # delete_tag에 해당하는 단어 쌍 지우기 (조사에 해당하는 단어 지우기)
            for key in list(dic.keys()):
                if dic[key] in delete_tag:
                    del dic[key]

        combine_word = [''.join(list(dic.keys())) for dic in dict_list] # 형태소로 분리된 각 단어 합치기
        result = ' '.join(combine_word) # 띄어쓰기로 분리된 각 토큰 합치기

        return result # 온전한 문장을 반환
    
    
    
    
    df = pd.read_csv("C:/Users/Soyoung Cho/Desktop/NMT Project/dataset/datalist_striped.csv")
    KOR_data = df['Korean']
    
    f = open("kor_no_josa.txt", "w", encoding = 'utf-8')
    for row in KOR_data[:100000]:
        f.write(remove_josa(row)) # 조사 제거한 문장 저장
        f.write('\n')
    f.close()
    
    spm.SentencePieceTrainer.Train('--input=kor_no_josa.txt \
                               --model_prefix=revise \
                               --vocab_size=100000 \
                               --hard_vocab_limit=false')
    
    sp = spm.SentencePieceProcessor()
    sp.Load('revise.model')
    print("success")
    sentence_piece = sp.EncodeAsPieces(x)
    
    return sentence_piece
 
    


In [ ]:
def English_tokenizer():
    data = pd.read_csv("C:/Users/Soyoung Cho/Desktop/NMT Project/dataset/datalist_striped.csv")
    ENG_data = data['English']

    f = open("eng.txt", "w", encoding = 'utf-8')
    for row in ENG_data[:100000]:
        f.write(row)
        f.write('\n')
        f.close()
    spm.SentencePieceTrainer.Train('--input=eng.txt \
                           --model_prefix=revise \
                           --vocab_size=100000 \
                           --hard_vocab_limit=false')
    sp = spm.SentencePieceProcessor()
    sp.Load('revise.model')
    return lambda x: sp.EncodeAsPieces(x)

In [18]:
sentence = "너 아침밥 먹음?"
m = MeCab.Tagger()
delete_tag = ['BOS/EOS', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']

sentence_split = sentence.split() # 원본 문장 띄어쓰기로 분리

dict_list = []

for token in sentence_split: # 띄어쓰기로 분리된 각 토큰 {'단어':'형태소 태그'} 와 같이 딕셔너리 생성
    m.parse('')
    node = m.parseToNode(token)
    word_list = []
    pos_list = []
    while node:
        morphs = node.feature.split(',')
        word_list.append(node.surface)
        pos_list.append(morphs[0])
        node = node.next
    dict_list.append(dict(zip(word_list, pos_list)))   
    
for dic in dict_list: # delete_tag에 해당하는 단어 쌍 지우기 (조사에 해당하는 단어 지우기)
    for key in list(dic.keys()):
        if dic[key] in delete_tag:
            del dic[key]

combine_word = [''.join(list(dic.keys())) for dic in dict_list] # 형태소로 분리된 각 단어 합치기
result = ' '.join(combine_word) # 띄어쓰기로 분리된 각 토큰 합치기

In [19]:
dict_list

[{'너': 'NP'}, {'아침밥': 'NNG'}, {'먹': 'VV', '음': 'ETN', '?': 'SF'}]

In [5]:
result

'안녕하세요. 감사합니다'

In [6]:
m = MeCab.Tagger()
delete_tag = ['BOS/EOS', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']
'''
"JKS":     "주격 조사",
"JKC":     "보격 조사",
"JKG":     "관형격 조사",
"JKO":     "목적격 조사",`
"JKB":     "부사격 조사",
"JKV":     "호격 조사",
"JKQ":     "인용격 조사",
"JX":      "보조사",
"JC":      "접속 조사",
'''

def remove_josa(sentence):
    sentence_split = sentence.split() # 원본 문장 띄어쓰기로 분리
    
    dict_list = []
    
    for token in sentence_split: # 띄어쓰기로 분리된 각 토큰 {'단어':'형태소 태그'} 와 같이 딕셔너리 생성
        m.parse('')
        node = m.parseToNode(token)
        word_list = []
        pos_list = []
        while node:
            morphs = node.feature.split(',')
            word_list.append(node.surface)
            pos_list.append(morphs[0])
            node = node.next
        dict_list.append(dict(zip(word_list, pos_list)))        

    for dic in dict_list: # delete_tag에 해당하는 단어 쌍 지우기 (조사에 해당하는 단어 지우기)
        for key in list(dic.keys()):
            if dic[key] in delete_tag:
                del dic[key]
    
    combine_word = [''.join(list(dic.keys())) for dic in dict_list] # 형태소로 분리된 각 단어 합치기
    result = ' '.join(combine_word) # 띄어쓰기로 분리된 각 토큰 합치기

    return result # 온전한 문장을 반환
        

예시 ) 너는  

--node--  
morphs: ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*']  
word_list :  ['']  
pos_list :  ['BOS/EOS']  

--node--  
morphs: ['NP', '*', 'F', '너', '*', '*', '*', '*']  
word_list :  ['', '너']  
pos_list :  ['BOS/EOS', 'NP']  

--node--  
morphs: ['JX', '*', 'T', '는', '*', '*', '*', '*']  
word_list :  ['', '너', '는']  
pos_list :  ['BOS/EOS', 'NP', 'JX']  

--node--  
morphs: ['BOS/EOS', '*', '*', '*', '*', '*', '*', '*']  
word_list :  ['', '너', '는', '']    
pos_list :  ['BOS/EOS', 'NP', 'JX', 'BOS/EOS']  
dict_list,  [{'': 'BOS/EOS', '너': 'NP', '는': 'JX'}]  

## sentencepiece 적용 (original과 조사 제거 버전 비교)

In [3]:
KOR_data = pd.read_csv("C:/Users/Soyoung Cho/Desktop/NMT Project/dataset/datalist_striped.csv", encoding = 'utf-8-sig')

In [5]:
KOR_data = KOR_data['Korean']

In [6]:
f1 = open("original.txt", "w", encoding = 'utf-8')
f2 = open("no_josa.txt", "w", encoding = 'utf-8')

for row in KOR_data[:100000]:
    f1.write(row) # 조사 제거 안한 원본 문장 저장
    f1.write('\n')
    
    f2.write(remove_josa(row)) # 조사 제거한 문장 저장
    f2.write('\n')
#f.close()
#f2.close()

In [7]:
# original 모델 생성 (조사 제거안한 문장으로 학습)
spm.SentencePieceTrainer.Train('--input=original.txt \
                               --model_prefix=original \
                               --vocab_size=100000 \
                               --hard_vocab_limit=false')

# 조사 제거한 문장으로 모델 생성
spm.SentencePieceTrainer.Train('--input=no_josa.txt \
                               --model_prefix=revise \
                               --vocab_size=100000 \
                               --hard_vocab_limit=false')

True

In [8]:
#모델 불러오기
sp1 = spm.SentencePieceProcessor()
sp1.Load('original.model')

sp2 = spm.SentencePieceProcessor()
sp2.Load('revise.model')

True

In [9]:
# input setence 
#sentence = "시 관계자는 선별진료소, 의료기관 등에 개인보호구인 덧신, 장갑·고글, 마스크 800개씩을 배포했지만 물량이 턱없이 부족하다며 정부에 추가 지원을 요청했다."
sentence = "안녕하세요. 감사합니다"

# 0. 원본 문장
print("<원본 문장>")
print(sentence+'\n')

# 1. original 모델 (조사 제거 안한 문장으로 학습)
print("<original 모델>")
print(sp1.EncodeAsPieces(sentence))

# 1. 조사 제거 모델  (조사 제거 한 문장으로 학습)
print("\n<조사 제거 모델>")
print(sp2.EncodeAsPieces(sentence))


<원본 문장>
안녕하세요. 감사합니다

<original 모델>
['▁안녕하세요', '.', '▁감사합니다']

<조사 제거 모델>
['▁안녕하세요', '.', '▁감사합니다']
